# Howto Polars
- Venis de **Pandas** y querés aprender a usar **Polars**
- La diferencia mas importante:
   - Polars establece operaciones que recién son ejecutadas cuando tienen un contexto (algo que en **Pandas** no existe).
   - Por tanto, uno opera con columnas genéricas de ningún dataframe (`pl.col`) en particular
   - Luego, cuando se les da contexto, es decir, se las vincula con un dataframe concreto, ahí se ejecutan y toman sentido ([Ver ejemplo de la documentación oficial](https://docs.pola.rs/user-guide/concepts/expressions-and-contexts/#expressions)).

In [1]:
import polars as pl
import pandas as pd

In [2]:
pl.__version__

'1.10.0'

In [3]:
pd.__version__

'2.2.3'

## Lectura de CSV  

In [4]:
df_pd = pd.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv")
df_pd.head()

,datetime,timelinux,s1
0,2022-11-17 17:04:39,1.668705,104.0
1,2022-11-17 17:34:37,1.668706,107.0
2,2022-11-17 18:04:38,1.668708,3.0
3,2022-11-17 18:34:38,1.668710,3.0
4,2022-11-17 19:04:37,1.668712,2.0


In [5]:
# En pandas con .info() conocemos el tipo de dato de cada columna 
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   1130 non-null   object 
 1   timelinux  1130 non-null   float64
 2   s1         1130 non-null   float64
dtypes: float64(2), object(1)
memory usage: 26.6+ KB


In [6]:
df_pl = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv")
# En polars ya nos informa el tipo de dato de cada columna
df_pl.head()

datetime,timelinux,s1
str,f64,f64
"""2022-11-17 17:04:39""",1.668705,104.0
"""2022-11-17 17:34:37""",1.668706,107.0
"""2022-11-17 18:04:38""",1.668708,3.0
"""2022-11-17 18:34:38""",1.66871,3.0
"""2022-11-17 19:04:37""",1.668712,2.0


In [7]:
# Polars con .schema nos informacada columna el tipo de dato asociado
df_pl.schema

Schema([('datetime', String), ('timelinux', Float64), ('s1', Float64)])

### Columna como datetime

- **Pandas** indica que las columnas datetime con ̣`parse_dates`

In [9]:
df_pd = pd.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", parse_dates = ["datetime"])
df_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130 entries, 0 to 1129
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   1130 non-null   datetime64[ns]
 1   timelinux  1130 non-null   float64       
 2   s1         1130 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 26.6 KB


- **Polars** tiene `try_parse_dates` que se da cuenta del formato y ya lo interpreta correctamente, fijate: 

In [10]:
df_pl = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", try_parse_dates=True)
df_pl.head()

datetime,timelinux,s1
datetime[μs],f64,f64
2022-11-17 17:04:39,1.668705,104.0
2022-11-17 17:34:37,1.668706,107.0
2022-11-17 18:04:38,1.668708,3.0
2022-11-17 18:34:38,1.66871,3.0
2022-11-17 19:04:37,1.668712,2.0


- Sino en **Polars** se lo indica manualmente con `schema_overrides` (antes era `dtypes`) así (ojo, tiene que ser compatible, sino se rompe):

In [32]:
df_pl = pl.read_csv("data/dataset_01/eqB4-s1-ciclo1.csv", 
                     schema_overrides={"datetime": pl.Datetime})
df_pl.head()

datetime,timelinux,s1
datetime[μs],f64,f64
2022-11-17 17:04:39,1.668705,104.0
2022-11-17 17:34:37,1.668706,107.0
2022-11-17 18:04:38,1.668708,3.0
2022-11-17 18:34:38,1.66871,3.0
2022-11-17 19:04:37,1.668712,2.0


## Seleccionar columnas 
- En **Pandas** seleccionar una columna te retorna una Serie, en **Polars**? también.
- En **Polars** también se puede usar el operador corchetes aunque el método .select() es preferido porque permite transformaciones complejas, encadenamiento, etc.

In [33]:
type(df_pd["datetime"])

pandas.core.series.Series

In [34]:
type(df_pl["datetime"])

polars.series.series.Series

- Seleccionamos varias columnas: igual que **Pandas**

In [35]:
df_pl[["datetime", "s1"]].head(3)

datetime,s1
datetime[μs],f64
2022-11-17 17:04:39,104.0
2022-11-17 17:34:37,107.0
2022-11-17 18:04:38,3.0


- Veamos con `.select()`

In [36]:
df_pl.select(['datetime', 's1']).head(3)

datetime,s1
datetime[μs],f64
2022-11-17 17:04:39,104.0
2022-11-17 17:34:37,107.0
2022-11-17 18:04:38,3.0


- También podemos usar argumentos posicionales con `pl.col()`:

In [37]:
df_pl.select(pl.col("datetime"), pl.col("s1") + 1000).head(3)

datetime,s1
datetime[μs],f64
2022-11-17 17:04:39,1104.0
2022-11-17 17:34:37,1107.0
2022-11-17 18:04:38,1003.0


- Vamos con algo un poco mas sofisticado, una columna que asigne un valor tope a aquellos que superen un umbral y -3.14 al resto, y denominemos a esta columna clamp:

In [38]:
df_pl.select(
    clamp = pl.when(pl.col("s1")>100).then(100).otherwise(-3.14)
)

clamp
f64
100.0
100.0
-3.14
-3.14
-3.14
…
100.0
-3.14
-3.14


Y si al resultado de esa operación la quiero como una nueva columna en mi dataframe?

In [41]:
df_pl = df_pl.with_columns(
    pl.when(pl.col("s1")>100)
    .then(100)
    .otherwise(-3.14)
    .alias("clamp") 
)

In [42]:
df_pl

datetime,timelinux,s1,clamp
datetime[μs],f64,f64,f64
2022-11-17 17:04:39,1.668705,104.0,100.0
2022-11-17 17:34:37,1.668706,107.0,100.0
2022-11-17 18:04:38,1.668708,3.0,-3.14
2022-11-17 18:34:38,1.66871,3.0,-3.14
2022-11-17 19:04:37,1.668712,2.0,-3.14
…,…,…,…
2022-12-11 04:34:25,1.670733,103.0,100.0
2022-12-11 05:04:25,1.670735,91.0,-3.14
2022-12-11 05:34:25,1.670737,86.0,-3.14


Conclusiones apresuradas: 
- es mejor hacer las cosas al modo **Polars**
- el `.select()` es poderoso y nos crea un nuevo dataframe
- para agregar columnas a un dataframe existente olvidamos el `.select()`, ahí juega `.with_columns()`

## No tan distintos

### Filtrado

Es parecido al **boolean indexing** de pandas solamente que se hace en el contexto de `.filter()` y se aplica a las columnas usando `pd.col("col_name")`

In [24]:
# Creamos un DataFrame a partir de una lista de diccionarios
data = [
    {"nombre": "Ana", "edad": 30, "ciudad": "Buenos Aires"},
    {"nombre": "Pedro", "edad": 25, "ciudad": "Córdoba"},
    {"nombre": "María", "edad": 35, "ciudad": "Buenos Aires"},
    {"nombre": "Emiliano", "edad": 43, "ciudad": "Tostado", "dni":"28331969"},
    
]

df_pl = pl.DataFrame(data)

# Mostramos el DataFrame
print(df_pl)

shape: (4, 4)
┌──────────┬──────┬──────────────┬──────────┐
│ nombre   ┆ edad ┆ ciudad       ┆ dni      │
│ ---      ┆ ---  ┆ ---          ┆ ---      │
│ str      ┆ i64  ┆ str          ┆ str      │
╞══════════╪══════╪══════════════╪══════════╡
│ Ana      ┆ 30   ┆ Buenos Aires ┆ null     │
│ Pedro    ┆ 25   ┆ Córdoba      ┆ null     │
│ María    ┆ 35   ┆ Buenos Aires ┆ null     │
│ Emiliano ┆ 43   ┆ Tostado      ┆ 28331969 │
└──────────┴──────┴──────────────┴──────────┘


In [25]:
# filtramos parecido a lo que serìa el boolean indexing de pandas

df_pl_filtrado = df_pl.filter(
    (pl.col("edad") >= 30) & (pl.col("ciudad") == "Buenos Aires")
)

df_pl_filtrado

nombre,edad,ciudad,dni
str,i64,str,str
"""Ana""",30,"""Buenos Aires""",null
"""María""",35,"""Buenos Aires""",null


In [27]:
# tenemos metodos sobre las columnas cuando tenemos valores nulos

df_pl_filtrado = df_pl.filter(
    (pl.col("edad") >= 30) & (~pl.col("dni").is_null())
)

df_pl_filtrado

nombre,edad,ciudad,dni
str,i64,str,str
"""Emiliano""",43,"""Tostado""","""28331969"""


## Iteración por filas
La prueba de que no es conveniente iterar por filas está en lo incómodo que es, tanto para **Pandas** como para **Polars**. Perdemos la capacidad vectorizada y de paralelización. Sin embargo a veces lo necesitamos. 

In [9]:
for row in df.iter_rows(named=True):
    print(row['nombre'])

Ana
Pedro
María
Emiliano


### Problema 1: procesamiento externo
- Una columna con path a diferentes archivos txt
- a cada archivo se lo procesa usando una funión externa
- el retorno de la función en un valor que guardamos en una lista y luego será una nueva columna del df

In [44]:
import subprocess
# funcion externa
def procesar_archivo(path: str) -> int:  
    """cuenta cantidad de lineas de un archivo usando wc"""
    command = ["wc", "-l", path]
    resultado = subprocess.run(command, capture_output=True, text=True)
    num_lineas = int(resultado.stdout.split()[0])

    return num_lineas

In [ ]:
# Crear un DataFrame de ejemplo
data = {'autor': ['charly', 'charly', 'vilca', 'flaco', 'fontanarrosa'], 
        'path': ['data/dataset_03/cancion_alicia.txt', 
                 'data/dataset_03/desarma_y_sangra.txt', 
                 'data/dataset_03/nada_tengo.txt', 
                 'data/dataset_03/quedandote_o_yendote.txt', 
                 'data/dataset_03/y_te_digo_mas.txt'], }
df = pl.DataFrame(data)
df

- **Enfoque iterativo**

In [68]:
%%time
# lista inicializada
wc = [0]*len(df)
# iteramos dataframe
for i,row in enumerate(df.iter_rows(named=True)):
    # por cada elemento del df, invocamos f y guardamos en lista
    wc[i] = procesar_archivo(row['path'])

# esa lista ahora es columna del df

df = df.with_columns(
    pl.Series("cant_lineas", wc, dtype=pl.Int32)
)
df

CPU times: user 0 ns, sys: 3.71 ms, total: 3.71 ms
Wall time: 6.73 ms


autor,path,cant_lineas,qty_lineas
str,str,i32,i64
"""charly""","""data/dataset_03/cancion_alicia…",32,32
"""charly""","""data/dataset_03/desarma_y_sang…",23,23
"""vilca""","""data/dataset_03/nada_tengo.txt""",34,34
"""flaco""","""data/dataset_03/quedandote_o_y…",27,27
"""fontanarrosa""","""data/dataset_03/y_te_digo_mas.…",79,79


- **Enfoque vectorial**

In [67]:
%%time
df = df.with_columns([
    pl.col("path")
    .map_elements(procesar_archivo, return_dtype=pl.Int64)
    .alias("qty_lineas")
])

print(df)

shape: (5, 4)
┌──────────────┬─────────────────────────────────┬─────────────┬────────────┐
│ autor        ┆ path                            ┆ cant_lineas ┆ qty_lineas │
│ ---          ┆ ---                             ┆ ---         ┆ ---        │
│ str          ┆ str                             ┆ i32         ┆ i64        │
╞══════════════╪═════════════════════════════════╪═════════════╪════════════╡
│ charly       ┆ data/dataset_03/cancion_alicia… ┆ 32          ┆ 32         │
│ charly       ┆ data/dataset_03/desarma_y_sang… ┆ 23          ┆ 23         │
│ vilca        ┆ data/dataset_03/nada_tengo.txt  ┆ 34          ┆ 34         │
│ flaco        ┆ data/dataset_03/quedandote_o_y… ┆ 27          ┆ 27         │
│ fontanarrosa ┆ data/dataset_03/y_te_digo_mas.… ┆ 79          ┆ 79         │
└──────────────┴─────────────────────────────────┴─────────────┴────────────┘
CPU times: user 4.42 ms, sys: 123 μs, total: 4.54 ms
Wall time: 7.51 ms


In [46]:
# Filtrar por etiqueta (por ejemplo, 'A')
df_filtered = df.filter(pl.col("autor") == 'charly')

df_filtered = df_filtered.with_columns([
    pl.col("path")
    .map_elements(procesar_archivo, return_dtype=pl.Int64)
    .alias("qty_lineas")
])

print(df_filtered)

shape: (2, 3)
┌────────┬─────────────────────────────────┬────────────┐
│ autor  ┆ path                            ┆ qty_lineas │
│ ---    ┆ ---                             ┆ ---        │
│ str    ┆ str                             ┆ i64        │
╞════════╪═════════════════════════════════╪════════════╡
│ charly ┆ data/dataset_03/cancion_alicia… ┆ 32         │
│ charly ┆ data/dataset_03/desarma_y_sang… ┆ 23         │
└────────┴─────────────────────────────────┴────────────┘
